In [1]:
import pandas as pd
import pickle
import random

## Reading CSV 

In [2]:
clusters = pd.read_csv('cluster_df.csv')
clusters = clusters.drop('Unnamed: 0', axis= 1)
clusters

,title,artist,no_cluster
0,You're The One - Un mal pour un bien,Petula Clark,6
1,Doctor My Eyes,Jackson Browne,1
2,Falling in Love at a Coffee Shop,Landon Pigg,4
3,Peaceful Easy Feeling - 2013 Remaster,Eagles,1
4,Summertime Blues,Eddie Cochran,5
...,...,...,...
5465,"Tighter, Tighter",Alive 'N Kickin',5
5466,Lunatic Fringe,Red Rider,7
5467,Run Runaway,Slade,5
5468,Hey Little Girl,Dee Clark,1


In [3]:
songs = pd.read_csv('100songs.csv')
songs = songs.drop('Unnamed: 0', axis= 1)
songs

,title,artist
0,Fast Car,Luke Combs
1,Last Night,Morgan Wallen
2,Need A Favor,Jelly Roll
3,Take Two,BTS
4,Save Me (with Lainey Wilson),Jelly Roll
...,...,...
95,Perfect,Ed Sheeran
96,Always Remember Us This Way,Lady Gaga
97,Whiskey Glasses,Morgan Wallen
98,She Had Me At Heads Carolina,Cole Swindell


In [4]:
songs

,title,artist
0,Fast Car,Luke Combs
1,Last Night,Morgan Wallen
2,Need A Favor,Jelly Roll
3,Take Two,BTS
4,Save Me (with Lainey Wilson),Jelly Roll
...,...,...
95,Perfect,Ed Sheeran
96,Always Remember Us This Way,Lady Gaga
97,Whiskey Glasses,Morgan Wallen
98,She Had Me At Heads Carolina,Cole Swindell


### Read pickle: kmeans and scaler

In [5]:
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
kmeans = pickle.load(open('kmeans.pkl', 'rb'))
scaler = pickle.load(open('scaler.pkl', 'rb'))

In [6]:
kmeans

KMeans(random_state=1234)

In [7]:
scaler

StandardScaler()

### Connection to Spotify

In [8]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

secrets_file = open("secrets.txt","r")

string = secrets_file.read()

secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

### Authentication

In [9]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

In [10]:
input_song = input("Enter a song name: ")
included_song = clusters['title'].isin([input_song]).any()
client_id=secrets_dict['clientid']
client_secret=secrets_dict['clientsecret']
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret'])) 

input_song = sp.search(q=input_song, type='track', limit=1)
uri = input_song['tracks']['items'][0]['uri']
audio_features = sp.audio_features(uri)

Enter a song name: Wannabe


In [11]:
audio_features

[{'danceability': 0.768,
  'energy': 0.864,
  'key': 11,
  'loudness': -6.145,
  'mode': 1,
  'speechiness': 0.0324,
  'acousticness': 0.101,
  'instrumentalness': 0,
  'liveness': 0.162,
  'valence': 0.891,
  'tempo': 110.008,
  'type': 'audio_features',
  'id': '1Je1IMUlBXcx1Fz0WE7oPT',
  'uri': 'spotify:track:1Je1IMUlBXcx1Fz0WE7oPT',
  'track_href': 'https://api.spotify.com/v1/tracks/1Je1IMUlBXcx1Fz0WE7oPT',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1Je1IMUlBXcx1Fz0WE7oPT',
  'duration_ms': 173027,
  'time_signature': 4}]

In [12]:
audio_features_df = pd.DataFrame(audio_features, index=[0])
audio_features_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.768,0.864,11,-6.145,1,0.0324,0.101,0,0.162,0.891,110.008,audio_features,1Je1IMUlBXcx1Fz0WE7oPT,spotify:track:1Je1IMUlBXcx1Fz0WE7oPT,https://api.spotify.com/v1/tracks/1Je1IMUlBXcx...,https://api.spotify.com/v1/audio-analysis/1Je1...,173027,4


In [13]:
selected_columns = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
audio_song = audio_features_df[selected_columns]
audio_song

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.768,0.864,-6.145,0.0324,0.101,0,0.162,0.891,110.008,173027,4


In [14]:
audio_song = scaler.transform(audio_song)
audio_song

array([[ 1.48562373,  1.23022092,  1.03960032, -0.34346236, -0.76285415,
        -0.29835514, -0.19451277,  1.1684356 , -0.41925968, -0.70769036,
         0.20158559]])

In [15]:
cluster_number = kmeans.predict(audio_song)

In [16]:
list(cluster_number)[0]

1

In [17]:
recommend_cluster = clusters[clusters['no_cluster']== list(cluster_number)[0]]

In [18]:
recommend_cluster['title']


1                              Doctor My Eyes
3       Peaceful Easy Feeling - 2013 Remaster
7                          Ain't That A Shame
12                                    Oh Boy!
13                 Lyin' Eyes - 2013 Remaster
                        ...                  
5456                             Lover Please
5458                            Mr. Big Stuff
5460                                 Jeopardy
5461                                Thank You
5468                          Hey Little Girl
Name: title, Length: 1570, dtype: object

In [19]:
recommend_cluster.sample()

,title,artist,no_cluster
1606,A Horse with No Name,America,1


In [20]:
def recommend_song():
    input_song = input("Enter a song name: ")
    #is_song_in_list = playlist_final['name'].isin([song]).any()
    client_id=secrets_dict['clientid']
    client_secret=secrets_dict['clientsecret']
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret'])) 
    if input_song in list(songs['title']):
        recommended_song = songs.sample(n=1)['title'].values[0]
        print("Your song is in the Top 100 list!")
        print("Here's another recommended song:", recommended_song)
    
    else:
        input_song = sp.search(q=input_song, type='track', limit=1)
        uri = input_song['tracks']['items'][0]['uri']
        audio_features = sp.audio_features(uri)
        audio_features_df = pd.DataFrame(audio_features, index=[0])
        selected_columns = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
        audio_song = audio_features_df[selected_columns]
        audio_song = scaler.transform(audio_song) 
        cluster_number = kmeans.predict(audio_song)
        recommend_cluster = clusters[clusters['no_cluster']== list(cluster_number)[0]]
        rec = recommend_cluster.sample()
        
        print("Here's another recommended song:", rec[['title','artist']])

In [21]:
recommend_song()

Enter a song name: Wannabe
Here's another recommended song:          title         artist
1953  Landfall  Jimmy Buffett
